# Dynamic Process: Robustness to the Distribution of Preference Intensities (C.2)

In [1]:
import pandas as pd
from poisson_approval import *

In [2]:
N_SAMPLES = 10000
N_MAX_EPISODES = 1000

In [3]:
distributions = {
    'U([0.09, .11])': RandProfileNoisyDiscreteUniform(
        types=[(ranking, 0.1, 0.01) for ranking in RANKINGS]),
    'U([0.89, .91])': RandProfileNoisyDiscreteUniform(
        types=[(ranking, 0.9, 0.01) for ranking in RANKINGS]),
}

## Condorcet Consistency

In [4]:
table_cond = pd.DataFrame()
table_cond.index.name = 'rho_o'
for distribution_name, distribution in distributions.items():
    rand_profile = RandConditional(
        distribution, 
        test=is_condorcet, n_trials_max=None
    )
    results = monte_carlo_fictitious_play(
        factory=rand_profile,
        n_samples=N_SAMPLES,
        n_max_episodes=N_MAX_EPISODES,
        voting_rules=VOTING_RULES,
        init='random_tau',
        monte_carlo_settings=[
            MCS_FREQUENCY_CW_WINS,
        ],
        file_save='sav/distribution_%s_with_CW.sav' % distribution_name,
    )
    for voting_rule in VOTING_RULES:
        table_cond.loc[distribution_name, voting_rule] = float(results[voting_rule]['mean_frequency_cw_wins'])
table_cond

,Approval,Plurality,Anti-plurality
rho_o,,,
"U([0.09, .11])",0.984800,0.6649,0.492624
"U([0.89, .91])",0.984975,0.6600,0.489420


## Welfare Loss

In [5]:
table_welfare_loss = pd.DataFrame()
table_welfare_loss.index.name = 'rho_o'
for distribution_name, distribution in distributions.items():
    results = monte_carlo_fictitious_play(
        factory=distribution,
        n_samples=N_SAMPLES,
        n_max_episodes=N_MAX_EPISODES,
        voting_rules=VOTING_RULES,
        init='random_tau',
        monte_carlo_settings=[
            MCS_WELFARE_LOSSES,
        ],
        file_save='sav/distribution_%s.sav' % distribution_name,
    )
    for voting_rule in VOTING_RULES:
        table_welfare_loss.loc[distribution_name, voting_rule] = float(
            results[voting_rule]['mean_utilitarian_welfare_loss'])
table_welfare_loss

,Approval,Plurality,Anti-plurality
rho_o,,,
"U([0.09, .11])",0.010898,0.085809,0.115399
"U([0.89, .91])",0.033006,0.061998,0.113344
